# Data Analytics Assignment

## Problem Statement

We are sharing data for our live demo classes right from parents showing interest in their conversion (Parent bought the live classes pack).

## Following are the details of the data:
1. Interested Users – Users (child) who have shown interest in joining the demo classes.
2. Booked Users – Users who booked a trail class on our portal. There are cases where duplicate trails are possible, count both.
3. Trainers – Trainers assigned to Trail Batch.
4. Subscription Bought – Details of Sales that are made. Use only those cases where sale value is greater than equal to 499/-.

## Using these data, create following metrics:
1. Sales conversion with respect to Trainers
2. Sales conversion with respect to Time slot
3. Sales conversion with respect to Topic
4. Sales conversion with respect to Source (Field available in Interested Users)
5. Sales conversion wrt grade
6. Time taken for Sales conversion
7. Time slot – attendance %

Sales conversion refers to the number of users who have bought the session after showing interest and attending the trial session.

In [1]:
# importing libraries

import pandas as pd
from pandas import ExcelWriter

import re

In [2]:
interested_users = pd.read_excel('assignment_data.xlsx', sheet_name = 'Interested Users')
booked_users = pd.read_excel('assignment_data.xlsx', sheet_name = 'Booked Users')
trainers = pd.read_excel('assignment_data.xlsx', sheet_name = 'Trainers')
subscription_bought = pd.read_excel('assignment_data.xlsx', sheet_name = 'Subscription Bought')

## Sales conversion with respect to Trainers

### Tidying trainer data

In [3]:
trainers.head(5)

,Trainer Name,Date,Timing,Grade,Batch size,Batch count,Batch,Topics
0,FAC_5001,16 April Thursday,11.15 am - 12 noon,"Grade 1,2",group,9.0,trial I,Rocking with rhymes
1,FAC_5002,16 April Thursday,11.15 am - 12 noon,"Grade 3,4",group,9.0,trial I,In my mind's eye
2,FAC_5003,16 April Thursday,11.15 am - 12 noon,"Grade 1,2",group,9.0,trial II,Word Whiz
3,FAC_5006,16 April Thursday,11.15 am - 12 noon,"Grade 3,4",group,7.0,trial II,Rocking with rhymes
4,FAC_5007,16 April Thursday,4:00-4:45 pm,"Grade 3,4",group,9.0,trial I,In my head or real


In [4]:
trainers['Topics'] = trainers['Topics'].str.strip()
trainers['Topics'] = trainers['Topics'].str.lower()

In [5]:
trainers['Start_Time'] = trainers['Timing'].apply(lambda x: re.findall('\d{1,2}(?:.|:)\d{2} ?(?:am|pm)?', x)[0]\
                                                  .strip().replace(" ","").replace('.',':'))

In [6]:
trainers.loc[trainers.Start_Time == '4:00','Start_Time'] = '04:00pm'
trainers.loc[trainers.Start_Time == '6:15','Start_Time'] = '06:15pm'
trainers.loc[trainers.Start_Time == '6:00pm','Start_Time'] = '06:00pm'

In [7]:
trainers['Start_Time'].unique()

array(['11:15am', '04:00pm', '06:15pm', '11:45am', '06:00pm'],
      dtype=object)

In [8]:
trainers['Grade'] = trainers['Grade'].str.strip()

In [9]:
trainers['Grade'].unique()

array(['Grade 1,2', 'Grade 3,4'], dtype=object)

### Tidying Booked Users data

In [10]:
booked_users.Batch[0]

'fREADom Trial Class - Grades 1, 2 [I] 11:15am - Thursday, April 16, 2020'

In [11]:
booked_users['Start_Time'] = booked_users['Batch'].apply(lambda x: re.findall('\d{2}:\d{2}(?:am|pm)',x)[0])

In [12]:
booked_users['Start_Time'].unique()

array(['11:15am', '04:00pm', '06:15pm', '11:45am', '06:00pm'],
      dtype=object)

In [13]:
booked_users['Grade'] = booked_users['Batch'].apply(lambda x: ",".join(re.findall('Grades \d, ?\d', x)[0]\
                                                                       .split(', ')).replace('Grades','Grade'))

In [14]:
booked_users['Grade'].unique()

array(['Grade 1,2', 'Grade 3,4'], dtype=object)

In [15]:
booked_users.head(2)

,Invitee UUID,Batch,Enrolment Type,Child Name,Grade,School,Phone,Parent Email,Opted Start Time,Opted Start Date,Facilitator UUID,Attendance,Start_Time
0,AGIV7AYTBF4ZA6SA,"fREADom Trial Class - Grades 1, 2 [I] 11:15am ...",Trial,CH_31440,"Grade 1,2",DPS,911100222445,s2m_test_21459@tests2m.com,16-04-2020 11:15:00,"11:15am - Thursday, April 16, 2020",AEFDOHX57EFVUS2G,Present,11:15am
1,HAPQZDZXDJPMFZ73,"fREADom Trial Class - Grades 1, 2 [I] 11:15am ...",Trial,CH_31441,"Grade 1,2",DPS,911100222446,s2m_test_21460@tests2m.com,16-04-2020 11:15:00,"11:15am - Thursday, April 16, 2020",AEFDOHX57EFVUS2G,Present,11:15am


### Merging Trainer data with Booked Users

In [16]:
trainers_booked_users = pd.merge(trainers[['Trainer Name','Start_Time','Grade']],
                                 booked_users[['Invitee UUID','Child Name','Start_Time','Grade']],
                                 on=['Start_Time','Grade'], how = 'left')

In [17]:
trainers_booked_users.head()

,Trainer Name,Start_Time,Grade,Invitee UUID,Child Name
0,FAC_5001,11:15am,"Grade 1,2",AGIV7AYTBF4ZA6SA,CH_31440
1,FAC_5001,11:15am,"Grade 1,2",HAPQZDZXDJPMFZ73,CH_31441
2,FAC_5001,11:15am,"Grade 1,2",AHLT5B5XGJTONPE3,CH_31442
3,FAC_5001,11:15am,"Grade 1,2",ABJQYBYVFXMAMETC,CH_31446
4,FAC_5001,11:15am,"Grade 1,2",AHKS3NCLBXORSZUV,CH_31112


In [18]:
trainers_users_split = trainers_booked_users\
.set_index(list(trainers_booked_users.columns[trainers_booked_users.columns != 'Grade']))\
.apply(lambda x: x.str.split(',').explode()).reset_index()

In [19]:
trainers_users_split['Grade'] = trainers_users_split.Grade.apply(lambda x: re.findall("\d",x)[0])

In [20]:
trainers_users_split.drop(['Invitee UUID','Start_Time'], axis = 1, inplace = True)
trainers_users_split.drop_duplicates(inplace = True)

In [21]:
trainers_users_split.head()

,Trainer Name,Child Name,Grade
0,FAC_5001,CH_31440,1
1,FAC_5001,CH_31440,2
2,FAC_5001,CH_31441,1
3,FAC_5001,CH_31441,2
4,FAC_5001,CH_31442,1


### Merging with subscription bought data

In [22]:
subscription_bought = subscription_bought[subscription_bought.Pricing >= 499]

In [23]:
subscription_bought = subscription_bought.reset_index(drop = True).reset_index()\
.rename(columns = {'index':'sub_id'})

In [24]:
subscription_bought.head()

,sub_id,Child Name,Registered Number,Registered Email ID,Payment date,Sale by,Grade,Pricing,Duration,Sessions preferred,Live session details
0,0,CH_31858,9.111002e+11,s2m_test_21900@tests2m.com,18-03-2020 12:00:00 AM,NaN,2,999,4G+1I,NaN,Live - Upsell (Larger plan)
1,1,CH_31860,9.111002e+11,s2m_test_21903@tests2m.com,18-03-2020 12:00:00 AM,NaN,4,1999,12G,NaN,Live - Upsell (Larger plan)
2,2,CH_31868,9.111002e+11,s2m_test_21911@tests2m.com,19-03-2020 12:00:00 AM,NaN,1,1999,10G+2I,NaN,Live - Upsell (Larger plan)
3,3,CH_31872,9.111002e+11,s2m_test_21915@tests2m.com,19-03-2020 12:00:00 AM,SP_1001,2,1999,10G+2I,NaN,Live - Upsell (Larger plan)
4,4,CH_31876,9.111002e+11,s2m_test_21919@tests2m.com,20-03-2020 12:00:00 AM,NaN,1,999,4G+1I,NaN,Live - Upsell (Larger plan)


### Calculating Sales Conversion

In [25]:
# checking which child bought package and got a trial training
trainer_conversion = pd.merge(trainers_users_split[['Trainer Name', 'Child Name']], 
                              subscription_bought[['sub_id','Child Name']], 
                              on = ['Child Name'], how = 'left')

In [26]:
trainer_child_conversion = trainer_conversion[~trainer_conversion.sub_id.isna()]\
.groupby('Trainer Name')['Child Name'].nunique().reset_index().rename(columns = {'Child Name':'Num_Sales'})

In [27]:
trainer_total_child = trainer_conversion\
.groupby('Trainer Name')['Child Name'].nunique().reset_index().rename(columns = {'Child Name':'Total_Child'})

In [28]:
trainer_sales_conversion = pd.merge(trainer_child_conversion, trainer_total_child, on = 'Trainer Name')

In [29]:
trainer_sales_conversion['Conversion_Rate'] = trainer_sales_conversion['Num_Sales']\
/trainer_sales_conversion['Total_Child']*100

In [30]:
# dropping cancelled trainer
trainer_sales_conversion.drop(trainer_sales_conversion[trainer_sales_conversion['Trainer Name'] == 'CANCELLED']\
                             .index, inplace = True)

In [31]:
trainer_sales_conversion.head()

,Trainer Name,Num_Sales,Total_Child,Conversion_Rate
1,FAC_5001,7,93,7.526882
2,FAC_5002,53,1098,4.826958
3,FAC_5003,16,189,8.465608
4,FAC_5004,12,199,6.030151
5,FAC_5006,31,667,4.647676


## Sales conversion with respect to Time-Slot

In [32]:
booked_subscribed = pd.merge(booked_users[['Child Name','Start_Time']], 
                             subscription_bought[['sub_id','Child Name']], 
                             on = 'Child Name', 
                             how = 'left')

In [33]:
slot_child_conversion = booked_subscribed[~booked_subscribed.sub_id.isna()]\
.groupby('Start_Time')['Child Name'].nunique().reset_index().rename(columns = {'Child Name':'Num_Sales'})

In [34]:
slot_total_child = booked_subscribed\
.groupby('Start_Time')['Child Name'].nunique().reset_index().rename(columns = {'Child Name':'Total_Child'})

In [35]:
slot_sales_conversion = pd.merge(slot_child_conversion, slot_total_child, on = 'Start_Time')

In [36]:
slot_sales_conversion['Conversion_Rate'] = slot_sales_conversion['Num_Sales']\
/slot_sales_conversion['Total_Child']*100

In [37]:
slot_sales_conversion.head()

,Start_Time,Num_Sales,Total_Child,Conversion_Rate
0,04:00pm,13,234,5.555556
1,06:00pm,14,476,2.941176
2,06:15pm,19,230,8.260870
3,11:15am,16,189,8.465608
4,11:45am,18,458,3.930131


## Sales conversion with respect to Topic

In [38]:
trainer_user_topics = pd.merge(trainers_users_split, 
                               trainers[['Trainer Name','Topics']], on = 'Trainer Name', how = 'left')

In [39]:
trainer_user_topics.head()

,Trainer Name,Child Name,Grade,Topics
0,FAC_5001,CH_31440,1,rocking with rhymes
1,FAC_5001,CH_31440,2,rocking with rhymes
2,FAC_5001,CH_31441,1,rocking with rhymes
3,FAC_5001,CH_31441,2,rocking with rhymes
4,FAC_5001,CH_31442,1,rocking with rhymes


In [40]:
# checking which child bought package and got a trial training
trainer_topic_conversion = pd.merge(trainer_user_topics[['Trainer Name', 'Child Name', 'Topics']], 
                              subscription_bought[['sub_id','Child Name']], 
                              on = ['Child Name'], how = 'left')

In [41]:
topic_child_conversion = trainer_topic_conversion[~trainer_topic_conversion.sub_id.isna()]\
.groupby('Topics')['Child Name'].nunique().reset_index().rename(columns = {'Child Name':'Num_Sales'})

In [42]:
topic_total_child = trainer_topic_conversion\
.groupby('Topics')['Child Name'].nunique().reset_index().rename(columns = {'Child Name':'Total_Child'})

In [43]:
topic_sales_conversion = pd.merge(topic_child_conversion, topic_total_child, on = 'Topics')

In [44]:
topic_sales_conversion['Conversion_Rate'] = topic_sales_conversion['Num_Sales']\
/topic_sales_conversion['Total_Child']*100

In [45]:
topic_sales_conversion.head()

,Topics,Num_Sales,Total_Child,Conversion_Rate
0,category spin,69,1446,4.771784
1,contract it,6,214,2.803738
2,elements of a story,69,1446,4.771784
3,emotional energy,47,1005,4.676617
4,in my mind's eye,58,1281,4.527713


## Sales conversion with respect to Source

In [46]:
source_conversion = pd.merge(interested_users[['Child name','source']], 
                             subscription_bought[['sub_id','Child Name']], 
                             left_on = ['Child name'], right_on = ['Child Name'], how = 'left')

In [47]:
source_conversion.drop('Child Name', inplace = True, axis = 1)

In [48]:
source_child_conversion = source_conversion[~source_conversion.sub_id.isna()]\
.groupby('source')['Child name'].nunique().reset_index().rename(columns = {'Child name':'Num_Sales'})

In [49]:
source_total_child = source_conversion\
.groupby('source')['Child name'].nunique().reset_index().rename(columns = {'Child name':'Total_Child'})

In [50]:
source_sales_conversion = pd.merge(source_child_conversion, source_total_child, on = 'source')

In [51]:
source_sales_conversion['Conversion_Rate'] = source_sales_conversion['Num_Sales']\
/source_sales_conversion['Total_Child']*100

In [52]:
source_sales_conversion.head()

,source,Num_Sales,Total_Child,Conversion_Rate
0,T1KRMSMS,4,53,7.547170
1,TEB11SMS,5,106,4.716981
2,TEB12SMS,3,67,4.477612
3,TEB2ASMS,3,106,2.830189
4,TEB3SMS,2,174,1.149425


## Sales conversion with respect to Grade

In [53]:
grade_conversion = pd.merge(trainer_user_topics[['Child Name','Grade']], 
                              subscription_bought[['sub_id','Child Name']], 
                              on = ['Child Name'], how = 'left')

In [54]:
grade_child_conversion = grade_conversion[~grade_conversion.sub_id.isna()]\
.groupby('Grade')['Child Name'].nunique().reset_index().rename(columns = {'Child Name':'Num_Sales'})

In [55]:
grade_total_child = grade_conversion\
.groupby('Grade')['Child Name'].nunique().reset_index().rename(columns = {'Child Name':'Total_Child'})

In [56]:
grade_sales_conversion = pd.merge(grade_child_conversion, grade_total_child, on = 'Grade')

In [57]:
grade_sales_conversion['Conversion_Rate'] = grade_sales_conversion['Num_Sales']\
/grade_sales_conversion['Total_Child']*100

In [58]:
grade_sales_conversion.head()

,Grade,Num_Sales,Total_Child,Conversion_Rate
0,1,29,697,4.160689
1,2,29,697,4.160689
2,3,43,786,5.470738
3,4,43,786,5.470738


## Time taken for Sales conversion

In [59]:
booked_users['Start_Slot'] = pd.to_datetime(booked_users['Opted Start Date'], format = '%I:%M%p - %A, %B %d, %Y')

In [60]:
# get first booking of each customer
idx = booked_users.groupby('Child Name')['Start_Slot'].transform(min) == booked_users['Start_Slot']
first_booking = booked_users[idx][['Child Name','Start_Slot']].reset_index(drop = True)\
.rename(columns = {'Start_Slot':'First_Trail_Slot'})

In [61]:
first_booking.head()

,Child Name,First_Trail_Slot
0,CH_31440,2020-04-16 11:15:00
1,CH_31441,2020-04-16 11:15:00
2,CH_31442,2020-04-16 11:15:00
3,CH_31443,2020-04-16 11:15:00
4,CH_31444,2020-04-16 11:15:00


In [62]:
subscription_bought['Payment_Date'] = pd.to_datetime(subscription_bought['Payment date'])

In [63]:
idx = subscription_bought.groupby('Child Name')['Payment_Date'].transform(min) == subscription_bought['Payment_Date']
first_subscription = subscription_bought[idx][['Child Name','Payment_Date']].reset_index(drop = True)\
.rename(columns = {'Payment_Date':'First_Payment_Date'})

In [64]:
sales_time_conversion = pd.merge(first_booking, first_subscription, on = 'Child Name')

In [65]:
sales_time_conversion['Time_Taken'] = sales_time_conversion['First_Payment_Date']\
- sales_time_conversion['First_Trail_Slot']

In [66]:
sales_time_conversion.head()

,Child Name,First_Trail_Slot,First_Payment_Date,Time_Taken
0,CH_31112,2020-04-16 11:15:00,2020-04-16 14:40:23,0 days 03:25:23
1,CH_31475,2020-04-16 16:00:00,2020-03-29 00:00:00,-19 days +08:00:00
2,CH_31479,2020-04-16 16:00:00,2020-12-04 21:18:44,232 days 05:18:44
3,CH_31482,2020-04-16 16:00:00,2020-04-17 11:51:16,0 days 19:51:16
4,CH_31515,2020-04-16 18:15:00,2020-04-18 10:59:59,1 days 16:44:59


## Time slot – attendance %

In [67]:
trainers['Date'] = trainers['Date'].apply(lambda x: " ".join(['2020', x]))
trainers['Date'] = pd.to_datetime(trainers['Date'], format = "%Y %d %B %A")
trainers['Date'] = trainers['Date'].dt.date

In [68]:
booked_users['Start_Date'] = booked_users['Start_Slot'].dt.date

In [69]:
trainers_batch = pd.merge(trainers[['Trainer Name','Start_Time','Grade', 'Date']],
                          booked_users[['Child Name','Start_Time','Grade', 'Start_Date']],
                          left_on=['Start_Time','Grade','Date'], 
                          right_on=['Start_Time','Grade','Start_Date'],
                          how = 'left')

In [70]:
batch_attendance = trainers_batch.groupby(['Trainer Name','Start_Date','Start_Time','Grade'])['Child Name']\
.nunique().reset_index().rename(columns = {'Child Name':'Attendance'})

In [71]:
batch_attendance

,Trainer Name,Start_Date,Start_Time,Grade,Attendance
0,CANCELLED,2020-04-17,11:15am,"Grade 3,4",12
1,CANCELLED,2020-04-25,06:00pm,"Grade 1,2",26
2,CANCELLED,2020-04-27,11:45am,"Grade 3,4",28
3,FAC_5001,2020-04-16,11:15am,"Grade 1,2",16
4,FAC_5002,2020-04-16,06:15pm,"Grade 3,4",31
...,...,...,...,...,...
168,FAC_5051,2020-04-26,11:45am,"Grade 1,2",22
169,FAC_5052,2020-04-26,06:00pm,"Grade 1,2",13
170,FAC_5052,2020-04-26,11:45am,"Grade 1,2",22
171,FAC_5052,2020-04-27,11:45am,"Grade 3,4",28


In [72]:
interested_users.loc[interested_users.grade == 'Grade 1','Grade_Group'] = "Grade 1,2"
interested_users.loc[interested_users.grade == 'Grade 2','Grade_Group'] = "Grade 1,2"
interested_users.loc[interested_users.grade == 'Grade 3','Grade_Group'] = "Grade 3,4"
interested_users.loc[interested_users.grade == 'Grade 4','Grade_Group'] = "Grade 3,4"

In [73]:
child_interested_slot = pd.merge(interested_users[['Child name', 'Grade_Group']], 
                                 booked_users[['Child Name','Start_Time', 'Start_Date']],
                                 left_on = 'Child name', right_on = 'Child Name', how = 'left')\
.drop('Child Name', axis = 1).dropna().reset_index(drop = True)\
.rename(columns = {'Grade_Group':'Grade','Child name':'Child Name'})

In [74]:
child_expected = child_interested_slot.groupby(['Start_Date','Start_Time','Grade'])['Child Name']\
.nunique().reset_index().rename(columns = {'Child Name':'Interested'})

In [75]:
child_expected.head()

,Start_Date,Start_Time,Grade,Interested
0,2020-04-16,04:00pm,"Grade 1,2",3
1,2020-04-16,04:00pm,"Grade 3,4",4
2,2020-04-16,06:15pm,"Grade 1,2",3
3,2020-04-16,06:15pm,"Grade 3,4",6
4,2020-04-16,11:15am,"Grade 1,2",1


In [76]:
attendance_perct = pd.merge(batch_attendance, child_expected, on = ['Start_Date','Start_Time','Grade'])

In [77]:
attendance_perct = attendance_perct[attendance_perct['Trainer Name'] != 'CANCELLED']\
.dropna().reset_index(drop = True)

In [78]:
attendance_perct = attendance_perct.groupby('Start_Time')[['Attendance','Interested']].sum().reset_index()

In [79]:
attendance_perct['Turnout_Percent'] = attendance_perct['Attendance']/attendance_perct['Interested'] * 100

In [80]:
attendance_perct.head()

,Start_Time,Attendance,Interested,Turnout_Percent
0,04:00pm,579,407,142.260442
1,06:00pm,1753,1489,117.730020
2,06:15pm,696,423,164.539007
3,11:15am,481,297,161.952862
4,11:45am,1556,1341,116.032811


## Saving as Excel

In [81]:
writer = ExcelWriter('Metrics.xlsx')

In [82]:
sales_time_conversion['Time_Taken'] = sales_time_conversion.Time_Taken.astype(str)

In [83]:
trainer_sales_conversion.to_excel(writer, 'trainer_sales_conversion', index = False)
slot_sales_conversion.to_excel(writer, 'slot_sales_conversion', index = False)
topic_sales_conversion.to_excel(writer, 'topic_sales_conversion', index = False)
source_sales_conversion.to_excel(writer, 'source_sales_conversion', index = False)
grade_sales_conversion.to_excel(writer, 'grade_sales_conversion', index = False)
sales_time_conversion.to_excel(writer, 'time_take_sales_conversion', index = False)
attendance_perct.to_excel(writer, 'attendance', index = False)

In [84]:
writer.save()